In [1]:
df <- read.csv('mlbootcamp5_train.csv', sep=';')
head(df)

id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,18393,2,168,62,110,80,1,1,0,0,1,0
1,20228,1,156,85,140,90,3,1,0,0,1,1
2,18857,1,165,64,130,70,3,1,0,0,0,1
3,17623,2,169,82,150,100,1,1,0,0,1,1
4,17474,1,156,56,100,60,1,1,0,0,0,0
8,21914,1,151,67,120,80,2,2,0,0,0,0


In [2]:
library(dplyr)


Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



In [3]:
summary(df)

       id             age            gender         height     
 Min.   :    0   Min.   :10798   Min.   :1.00   Min.   : 55.0  
 1st Qu.:25007   1st Qu.:17664   1st Qu.:1.00   1st Qu.:159.0  
 Median :50002   Median :19703   Median :1.00   Median :165.0  
 Mean   :49972   Mean   :19469   Mean   :1.35   Mean   :164.4  
 3rd Qu.:74889   3rd Qu.:21327   3rd Qu.:2.00   3rd Qu.:170.0  
 Max.   :99999   Max.   :23713   Max.   :2.00   Max.   :250.0  
     weight           ap_hi             ap_lo           cholesterol   
 Min.   : 10.00   Min.   : -150.0   Min.   :  -70.00   Min.   :1.000  
 1st Qu.: 65.00   1st Qu.:  120.0   1st Qu.:   80.00   1st Qu.:1.000  
 Median : 72.00   Median :  120.0   Median :   80.00   Median :1.000  
 Mean   : 74.21   Mean   :  128.8   Mean   :   96.63   Mean   :1.367  
 3rd Qu.: 82.00   3rd Qu.:  140.0   3rd Qu.:   90.00   3rd Qu.:2.000  
 Max.   :200.00   Max.   :16020.0   Max.   :11000.00   Max.   :3.000  
      gluc           smoke              alco           

In [5]:
table(df$cardio)


    0     1 
35021 34979 

In [5]:
head(arrange(df, cardio, desc(ap_hi)))

id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
67502,19731,1,160,65,14020,90,1,1,0,0,1,0
79679,23182,1,161,105,13010,80,1,1,0,0,0,0
61618,20961,2,170,78,1400,90,2,1,0,0,1,0
81769,21948,2,166,73,1300,90,1,1,0,1,1,0
77010,18511,1,164,54,960,60,1,1,0,0,1,0
6822,14425,1,168,63,909,60,2,1,0,0,1,0


## Задание 1 

 Сколько мужчин и женщин представлено в этом наборе данных?

In [4]:
df %>% group_by(gender) %>%
  summarise(number = n(),
            mean.height = mean(height))


gender,number,mean.height
1,45530,161.3556
2,24470,169.9479


#### 1 - женщины, их 45530
#### 2 - мужчины, их 24470

In [7]:
table(df$gender)


    1     2 
45530 24470 

## Задание 2

Кто в среднем реже указывает, что употребляет алкоголь – мужчины или женщины?

In [18]:
df %>% group_by(gender) %>%
  select(gender, alco) %>%
  summarise_all(funs(mean))

gender,alco
1,0.02549967
2,0.10637515


**Женщины реже указывают, что употребляют алкоголь**

##  Задание 3

Во сколько раз округленно процент курящих среди мужчин больше, чем процент курящих среди женщин (по крайней мере, по этим анкетным данным)?

In [8]:
mean(filter(df, df['gender'] == 1)[, 'smoke'] ) -> smoke_wm
mean(filter(df, df['gender'] == 2)[, 'smoke'] ) -> smoke_m
round(smoke_m / smoke_wm)

[1] 12

**Примерно в 12 раз больше**

In [9]:
df %>% group_by(gender) %>%
  select(gender, smoke) %>%
  summarise_all(funs(mean))

gender,smoke
1,0.01785636
2,0.21888026


## Задание 4

В чём здесь измеряется возраст? 
На сколько месяцев (примерно) отличаются медианные значения возраста курящих и некурящих?

In [37]:
head(select(df, age))

age
18393
20228
18857
17623
17474
21914


Нетрудно догадаться, что возраст указан в днях

In [6]:
df %>% group_by(smoke) %>%
  select(smoke, age) %>%
  summarise_all(funs(median))

smoke,age
0,19722
1,19125


In [6]:
median(filter(df,df['smoke'] == 1)[, 'age']) / 30 -> smoke
median(filter(df,df['smoke'] == 0)[, 'age']) / 30 -> non_smoke
round(non_smoke - smoke)

[1] 20

На 20 месяцев отличаются медианные значения возраста курящих и некурящих

## Задание 5

Во сколько раз округленно отличаются доли больных людей (согласно целевому признаку, cardio) в двух подвыборках?

Первая: курящие мужчины от 60 до 64 лет включительно с верхним артериальным давлением строго меньше 120 мм рт.ст. и концентрацией холестерина – 4 ммоль/л

Вторая: мужчины того же возраста с верхним артериальным давлением от 160 (включительно) до 180 мм рт.ст. (не включительно) и концентрацией холестерина – 8 ммоль/л

In [7]:
mutate(df, age_years=round(age/365.25)) ->df1 #добавляем возраст в годах

In [8]:
smoking_old_men <- filter(df1, df1['gender'] == 2 & df1['age_years'] >= 60
                    & df1['age_years'] < 65 & df1['smoke'] == 1)
head(smoking_old_men)

id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,age_years
29,21755,2,162,56,120,70,1,1,1,0,1,0,60
52,23388,2,162,72,130,80,1,1,1,0,1,1,64
90,22099,2,171,97,150,100,3,1,1,0,1,1,61
240,21848,2,170,69,120,80,1,1,1,1,1,0,60
248,22671,2,174,74,130,80,3,1,1,0,1,1,62
318,23376,2,175,75,180,100,3,1,1,1,1,1,64


In [9]:
a <-mean(filter(smoking_old_men, smoking_old_men['cholesterol'] == 1 &
               smoking_old_men['ap_hi'] < 120)[, 'cardio'])

In [10]:
b <- mean(filter(smoking_old_men, smoking_old_men['cholesterol'] == 3 &
               smoking_old_men['ap_hi'] < 180 & smoking_old_men['ap_hi'] >= 160)[, 'cardio'])

In [11]:
round(b/a,2) 

[1] 3.28

**Ответ: примерно в 3 раза**

## Задание 6

Выберите верные утверждения:
- Медианный BMI по выборке превышает норму
- У женщин в среднем BMI ниже, чем у мужчин
- У здоровых в среднем BMI выше, чем у больных
- В сегменте здоровых и непьющих мужчин в среднем BMI ближе к норме, чем в сегменте здоровых и непьющих женщин

In [13]:
mutate(df1, BMI= weight / (height / 100) ** 2) ->df1 #Новый признак Body Mass Index

Нормальными считаются значения BMI от 18.5 до 25

In [14]:
median(df1[['BMI']]) 

[1] 26.37407

Медианный BMI по выборке превышает норму - первое утверждение верно

In [15]:
df1 %>% group_by(gender) %>%
  select(gender, BMI) %>%
  summarise_all(funs(mean)) 

gender,BMI
1,27.98758
2,26.75444


Второе утверждение неверно, у женщин в среднем BMI выше, чем у мужчин

In [16]:
df1 %>% group_by(cardio) %>%
  select(cardio, BMI) %>%
  summarise_all(funs(mean)) 

cardio,BMI
0,26.54818
1,28.56606


Третье утверждение неверно

In [17]:
women <- filter(df1, df1['gender'] == 1 & df1['cardio'] == 0 & df1['alco'] == 0)
men <- filter(df1, df1['gender'] == 2 & df1['cardio'] == 0 & df1['alco'] == 0)
mean(women[['BMI']])
mean(men[['BMI']])

[1] 26.84541

[1] 25.87264

In [18]:
df1 %>% group_by(gender, alco, cardio) %>%
  select(gender, alco, cardio, BMI) %>%
  summarise_all(funs(mean)) 

gender,alco,cardio,BMI
1,0,0,26.84541
1,0,1,29.05277
1,1,0,28.67146
1,1,1,30.81235
2,0,0,25.87264
2,0,1,27.52245
2,1,0,26.09722
2,1,1,28.22657


Утверждение 4 верно

## Задание 7 

Отфильтровать следующие сегменты пациентов (считаем это ошибками в данных):

- указанное нижнее значение артериального давления строго выше верхнего
- рост строго меньше 2.5%-перцентили или строго больше 97.5%-перцентили 
- вес строго меньше 2.5%-перцентили или строго больше 97.5%-перцентили

In [19]:
filtered_df <- filter(df1, df1['ap_lo'] <= df1['ap_hi'] & 
                 df1['height'] >= quantile(df1$height, 0.025)[[1]] &
                 df1['height'] <= quantile(df1$height, 0.975)[[1]] &
                 df1['weight'] >= quantile(df1$weight, 0.025)[[1]] & 
                 df1['weight'] <= quantile(df1$weight, 0.975)[[1]])


Сколько процентов данных округленно мы выбросили?

In [20]:
(1 - round(nrow(filtered_df) / nrow(df1), 2)) * 100

[1] 10

Ответ: выбросили около 10% данных